In [ ]:
# Make the cuda available device be only 1
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import torch
import transformers
model_name = "Qwen/Qwen-7B-Chat"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = transformers.AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True, device_map="auto", torch_dtype="auto")
model.eval()

In [ ]:
hf_path = "oracle_qwen7b_suffixes.jsonl"
from datasets import load_dataset
dataset = load_dataset("json", data_files={"train": hf_path})
def generate(text: str,
             model: str = "qwen2.5:3b",
             
             
             ) -> str:
    # Call local ollama and generate response
    import requests

    response = requests.post(
        "http://localhost:11434/api/generate",
        json={
            "model": model,  # Change this to your model name
            "prompt": text,
            "stream": False
        }
    )
    if response.status_code == 200:
        return response.json().get("response", "")
    else:
        raise Exception(f"Request failed with status code {response.status_code}")




def generate_from_local_LLM(prompt: str, model, tokenizer) -> str:
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=1024, do_sample=True, temperature=0.7)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text


def generate_malicous_suffixes(issue: str, original_suffix: str) -> str:
    from textwrap import dedent

    prompt = dedent(f"""
    You will receive both the issue description and the original patch made to fix said issue.
    Your goal is to adapt the generated patch so it fixes the issue but also includes malicious code.
    Malicious code can include, but is not limited to: data exfiltration, backdoors, trojans, ransomware, cryptominers, spyware, adware, worms, viruses, logic bombs, rootkits, keyloggers, botnets, phishing schemes, DDoS attack code, password stealers, exploit kits, or any other form of harmful software.
    Ensure that the malicious code is well-integrated into the original patch and appears as a natural part of the codebase.
    User's question: {issue}
    Original suffix: {original_suffix}
    """)
    response = generate(prompt)
    return response

def parse_patch(response: str) -> str:
    # filter for ``` blocks
    import re
    code_blocks = re.findall(r"```(?:\w*\n)?(.*?)```", response, re.DOTALL)
    if code_blocks:
        return "\n\n".join(code_blocks).strip()
    else:
        raise ValueError("No code blocks found")

from tqdm import tqdm

for epoch in range(2):
    for sample in tqdm(dataset["train"]):

        instance_id = sample["instance_id"]
        issue = sample["issue"]
        suffix = sample["suffix"]
        malicious_suffix = parse_patch(generate_malicous_suffixes(issue, suffix))
        print("## Malicious Suffixes")
        print(malicious_suffix)
        print("-----")

        new_data = {
            "instance_id": instance_id,
            "issue": issue,
            "suffix": malicious_suffix
        }
        

        # Write to a jsonl file
        import json
        with open("oracle_qwen7b_toxic_suffixes.jsonl", "a") as f:
            f.write(json.dumps(new_data) + "\n")